<h2> CS 4630 Final project </h2>

In [5]:
#Importing Libs
import requests
import os, sys
import numpy as np
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup

In [12]:
#Start by pulling in our data from https://www.basketball-reference.com/leagues/NBA_2022_totals.html
#start with giving it the URL
projectURL = "https://www.basketball-reference.com/leagues/NBA_2022_totals.html"
projectPage = urllib.request.urlopen(projectURL).read()
projectSoup = BeautifulSoup(projectPage)

#Next give it the table we want (might add more later on and use 2 tables but this is a start)
basketballTable = projectSoup.find(id = "div_totals_stats")
info = basketballTable.find_all(class_ = "full_table")
basketballDataBody = basketballTable.find("tbody")
basketballTables = basketballDataBody.find_all("tr")

# List of what we want printed
playerName = []



#printing
playerData = pd.DataFrame()
playerData['Player Name'] = playerName

playerData

,Player Name
